In [1]:
import torch
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

from CNN_ensemble import train_model as train_cnn, load_cnn_data as load_cnn_data
from LSTM import train_model as train_lstm, load_lstm_data as load_lstm_data

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

def evaluate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    accuracy = torch.sum(preds == labels).item() / len(preds)
    return accuracy * 100

def combine_models(cnn_model, lstm_model, test_loader_cnn, test_loader_lstm):
    cnn_model.eval()
    lstm_model.eval()
    combined_accuracy = 0
    with torch.no_grad():
        for (images, labels), (numerical, _) in zip(test_loader_cnn, test_loader_lstm):
            images, labels, numerical = images.to(device), labels.to(device), numerical.to(device)
            cnn_outputs = cnn_model(images)
            lstm_outputs = lstm_model(numerical)
            combined_outputs = (F.softmax(cnn_outputs, dim=1) + F.softmax(lstm_outputs, dim=1)) / 2
            combined_accuracy += evaluate_accuracy(combined_outputs, labels)
    combined_accuracy /= len(test_loader_cnn)
    return combined_accuracy

if __name__ == "__main__":
    # Train CNN and LSTM models
    cnn_model = train_cnn()
    lstm_model = train_lstm()


    # Load test data
    _, _, test_images, test_labels = load_cnn_data()
    test_numerical, test_labels = load_lstm_data()[2:]  

    # Prepare DataLoaders
    test_loader_cnn = DataLoader(TensorDataset(test_images, test_labels), batch_size=64, shuffle=False)
    test_loader_lstm = DataLoader(TensorDataset(test_numerical, test_labels), batch_size=64, shuffle=False)


    # Combine models and evaluate on test set
    combined_test_accuracy = combine_models(cnn_model, lstm_model, test_loader_cnn, test_loader_lstm)
    print(f'Combined Model Test Accuracy: {combined_test_accuracy:.2f}%')


Using cuda device
Using cuda device
Using cuda device
Epoch 1/15, Loss: 0.3976, Train Accuracy: 62.24%, Test Accuracy: 59.58%
Epoch 2/15, Loss: 0.0968, Train Accuracy: 66.95%, Test Accuracy: 62.25%
Epoch 3/15, Loss: 0.0579, Train Accuracy: 87.41%, Test Accuracy: 72.02%
Epoch 4/15, Loss: 0.0383, Train Accuracy: 87.11%, Test Accuracy: 72.33%
Epoch 5/15, Loss: 0.0382, Train Accuracy: 90.26%, Test Accuracy: 73.53%
Epoch 6/15, Loss: 0.0302, Train Accuracy: 97.80%, Test Accuracy: 74.06%
Epoch 7/15, Loss: 0.0263, Train Accuracy: 61.09%, Test Accuracy: 58.79%
Epoch 8/15, Loss: 0.0300, Train Accuracy: 89.60%, Test Accuracy: 68.39%
Epoch 9/15, Loss: 0.0164, Train Accuracy: 97.53%, Test Accuracy: 75.52%
Epoch 10/15, Loss: 0.0196, Train Accuracy: 99.05%, Test Accuracy: 76.81%
Epoch 11/15, Loss: 0.0009, Train Accuracy: 100.00%, Test Accuracy: 78.76%
Epoch 12/15, Loss: 0.0003, Train Accuracy: 100.00%, Test Accuracy: 78.69%
Epoch 13/15, Loss: 0.0002, Train Accuracy: 100.00%, Test Accuracy: 78.67%
Epo

c:\Users\chris\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/25, Loss: 1.0001, Train Acc: 40.83%, Test Acc: 39.83%
Epoch 2/25, Loss: 0.9716, Train Acc: 58.28%, Test Acc: 52.79%
Epoch 3/25, Loss: 0.8665, Train Acc: 59.38%, Test Acc: 53.48%
Epoch 4/25, Loss: 0.7948, Train Acc: 70.54%, Test Acc: 57.16%
Epoch 5/25, Loss: 0.6517, Train Acc: 77.73%, Test Acc: 61.27%
Epoch 6/25, Loss: 0.4979, Train Acc: 86.04%, Test Acc: 63.60%
Epoch 7/25, Loss: 0.3825, Train Acc: 92.70%, Test Acc: 64.48%
Epoch 8/25, Loss: 0.2089, Train Acc: 96.65%, Test Acc: 65.20%
Epoch 9/25, Loss: 0.2018, Train Acc: 96.04%, Test Acc: 64.42%
Epoch 10/25, Loss: 0.1100, Train Acc: 98.64%, Test Acc: 65.76%
Epoch 11/25, Loss: 0.0935, Train Acc: 99.24%, Test Acc: 66.14%
Epoch 12/25, Loss: 0.0739, Train Acc: 99.13%, Test Acc: 64.94%
Epoch 13/25, Loss: 0.0652, Train Acc: 99.60%, Test Acc: 66.56%
Epoch 14/25, Loss: 0.0534, Train Acc: 99.76%, Test Acc: 66.53%
Epoch 15/25, Loss: 0.0452, Train Acc: 98.78%, Test Acc: 65.22%
Epoch 16/25, Loss: 0.0386, Train Acc: 99.82%, Test Acc: 67.57%
E